 Import all necessary librarys

In [16]:
import numpy as np
import sys, os
import time
import matplotlib.pyplot as plt
import sqlite3 as sql

%matplotlib qt
plt.ion()

savedata = r'd:\Documenten\Documents\GitHub\Active-Vibration-Stabilization\Meetfiles\data\2023-04-17'

os.chdir(savedata)
print(os.getcwd())

files = os.listdir()
for i,f in enumerate(files):
    files[i] = f.split('.db',)[0].split('.npy')[0]
files = list(set(files))
files.sort()
f_min = 0.50
f_max = 1.70

d:\Documenten\Documents\GitHub\Active-Vibration-Stabilization\Meetfiles\data\2023-04-17


In [17]:
DISTANCES = {
'gnx':1.3750, 'gex':0.0000, 'gtx':0.0559,
'gny':0.0000, 'gey':1.3750, 'gty':0.3449,
'gnz':0.7528, 'gez':0.7528, 'gtz':3.2772,
}

AMAT = np.matrix([
#    x | y | z |        rx         |        ry         |          rz
    [0 , 1 , 0 , -DISTANCES['gnz'] ,                 0 , -DISTANCES['gnx']],    # northY
    [0 , 0 , 1 , -DISTANCES['gny'] ,  DISTANCES['gnx'] ,                 0],    # northZ
    [1 , 0 , 0 ,                 0 ,  DISTANCES['gez'] ,  DISTANCES['gey']],    # eastX
    [0 , 0 , 1 , -DISTANCES['gey'] ,  DISTANCES['gex'] ,                 0],    # eastZ
    [1 , 0 , 0 ,                 0 , -DISTANCES['gtz'] ,  DISTANCES['gty']],    # topX
    [0 , 1 , 0 ,  DISTANCES['gtz'] ,                 0 , -DISTANCES['gtx']],    # topY
])
AMAT_INV = np.linalg.inv(AMAT)

Loop through all Database files and timstamp files.</br>
and calculate the x,y,z,rx,ry,rz components</br>
Calculate the fourier transform of the z component</br>


```
data = [
    [t1, t2, t3, ..., tn],   # Array of time data
    [s1, s2, s3, ..., sn],   # Array of north Y
    [s1, s2, s3, ..., sn],   # Array of north Z
    [s1, s2, s3, ..., sn],   # Array of East  X
    [s1, s2, s3, ..., sn],   # Array of East  Z
    [s1, s2, s3, ..., sn],   # Array of Top   X
    [s1, s2, s3, ..., sn]    # Array of Top   Y
]
```
In this structure, the first array contains time data for each reading, while the other 6 arrays contain the sensor readings. The `n` represents the number of readings that are stored in the array. Each row of data represents a single reading, with the time and sensor readings for that particular reading stored in the corresponding columns.

To access a specific value in the data array, you would use the syntax `data[row_index,column_index]`. For example, to access the time value for the third reading, you would use `data[0,2]`, since the time data is stored in the first array and the third column. Similarly, to access the value of the East Z sensor reading for the fifth reading, you would use `data[4,5]`, since the fourth sensor reading is stored in the fifth row and fourth column of the array.


```
out = [
    [ x1,  x2,  x3, ...,  xn],   # Array of linear x movement
    [ y1,  y2,  y3, ...,  yn],   # Array of linear y movement
    [ z1,  z2,  z3, ...,  zn],   # Array of linear z movement
    [rx1, rx2, rx3, ..., rxn],   # Array of rotation around x
    [ry1, ry2, ry3, ..., ryn],   # Array of rotation around y
    [rz1, rz2, rz3, ..., rzn],   # Array of rotation around z
]
```

The 6xn NumPy array contains transformed values along the 6 degrees of freedom. The six degrees of freedom represent the ways in which an object can move or change orientation in three-dimensional space. These degrees of freedom include translation along the x, y, and z axes, as well as rotation around these same axes.

The array is structured such that each row represents a specific transformation applied to the object. The first column contains the translation along the x-axis, the second column contains the translation along the y-axis, and the third column contains the translation along the z-axis. The fourth, fifth, and sixth columns contain the rotation around the x, y, and z axes, respectively.



In [40]:
try:
    os.mkdir('pictures')
except:
    print('Directory already exists')
transferFunction = np.empty((0,6,4),np.complex_)

fig1, fig1_axes = plt.subplots(1,1,sharex=True,gridspec_kw=dict(hspace=0))
fig1_manager = plt.get_current_fig_manager()
fig1_manager.set_window_title(f'{f:.2f}')
fig1_manager.window.showMaximized()

fig1_axes.set_xscale('log')
fig1_axes.set_yscale('linear')
fig1_axes.set_xlim(1e-3,1e2)
fig1_axes.set_ylim(-0.05, 1.5)
fig1_axes.set_xlabel('Frequencies $[Hz]\longrightarrow$')
fig1_axes.set_ylabel(r'Amplitude $[\frac{mm}{s}]\longrightarrow$')

plt.pause(0.002)
fig1.tight_layout()
plt.pause(0.002)

fig1_fourier, = fig1_axes.plot([], [], 'k')
fig1_max_point, = fig1_axes.plot([],[],'ro')


for f in np.arange(f_min,f_max+0.05, 0.05):
    file = f'2023-04-17 - 174804 ({f:.2f})'
    db = sql.connect(f'{file}.db')
    cur = db.cursor()

    timestamps = np.load(f'{file}.npy')

    print(f'{f:.2f}')
    timestamp = np.where(np.round(timestamps,2) == np.round(f,2))[0][0]
    t_min = timestamps[timestamp+1]
    t_max = timestamps[timestamp+2]

    print(f'f\t=\t{timestamps[timestamp]}\nt_min\t=\t{t_min}\nt_max\t=\t{ t_max}')

    meas0 = np.transpose(cur.execute(f'SELECT t,northY,northZ,eastX,eastZ,topX,topY FROM guralp WHERE t BETWEEN {timestamps[1]} AND {timestamps[2]} ').fetchall())
    meas0 = np.mat(meas0[1:])
    meas0 = np.array(AMAT_INV*meas0)

    data = np.transpose(cur.execute(f'SELECT t,northY,northZ,eastX,eastZ,topX,topY FROM guralp WHERE t BETWEEN {t_min} AND {t_max} ').fetchall())
    meas = np.mat(data[1:])
    out = np.array(AMAT_INV*meas)

    dt = np.average([h-l for h,l in zip(data[0][1:],data[0])])
    print((data[0,-1]-data[0,0])*f)
    temp_transferFunction = np.empty((0,4),np.complex_)
    for axis, meas0_axis,out_axis  in zip('x y z rx ry rz'.split(),meas0,out):
        fourier_transform0 = 2 * np.fft.fft(meas0_axis, len(meas0_axis))[range(int(len(out_axis)/2))] / len(meas0_axis)
        fourier_transform = 2 * np.fft.fft(out_axis, len(out_axis))[range(int(len(out_axis)/2))] / len(out_axis)
        fourier_frequencies = np.fft.fftfreq(len(out_axis),dt)[range(int(len(out_axis)/2))]

        Hz = np.argmin(np.abs((f * np.ones_like(fourier_frequencies)) - fourier_frequencies))  # `Hz` is the index of `f` in the array fourier_frequencies
        temp_transferFunction = np.append(temp_transferFunction, [[f, fourier_transform0[Hz], fourier_transform[Hz], fourier_transform0[Hz]-fourier_transform[Hz]]], 0)

        max_amp = np.max(np.abs(fourier_transform[(0.1 < fourier_frequencies) * (fourier_frequencies < 10)]))
        max_Hz = np.where(np.abs(fourier_transform) == max_amp)[0][0]
        freq = fourier_frequencies[max_Hz]
        amp = np.abs(fourier_transform[max_Hz])
        phase = np.rad2deg(np.angle(fourier_transform[max_Hz]))
    
        label = fr'''f={freq:.4f}[Hz] A = {amp:.2f} $\varphi$ = {phase:.2f} [rad]'''
        print(label)

        fig1_fourier.set_data(fourier_frequencies, np.abs(fourier_transform))
        fig1_axes.set_title(f'Fourier transform of the signal along the {axis}-axis', dict(fontsize=48))

        fig1_anno = fig1_axes.annotate(label,(freq,amp), textcoords="offset points", xytext=(30,10), ha='center')
        fig1_max_point.set_data([freq],[amp])

        plt.pause(0.002)
        fig1.tight_layout()
        plt.pause(0.002)

        title = f'Fourier Transform ({fig1_manager.get_window_title()} Hz, 2 V, {axis})'
        fig1.savefig(f'pictures/{title}.png', transparent=True,dpi=600,format='png')
        fig1_anno.remove()

        
    
        
    db.close()

    transferFunction = np.append(transferFunction, [temp_transferFunction], 0)
plt.close(fig1)
np.save(f'{savedata}\Full Transfer Function',transferFunction, True)

Directory already exists
0.50
f	=	0.5
t_min	=	2020.00439453125
t_max	=	4020.0048828125
1000.0
f=1.0000[Hz] A = 0.10 $\varphi$ = -108.72 [rad]
f=1.0000[Hz] A = 0.08 $\varphi$ = 83.07 [rad]
f=1.0000[Hz] A = 0.02 $\varphi$ = -111.11 [rad]
f=1.0000[Hz] A = 0.02 $\varphi$ = -164.48 [rad]
f=1.0000[Hz] A = 0.02 $\varphi$ = -136.36 [rad]
f=1.0000[Hz] A = 0.09 $\varphi$ = 67.57 [rad]
0.55
f	=	0.55
t_min	=	1838.1851806640625
t_max	=	3656.534912109375
1000.092352294922
f=1.0999[Hz] A = 0.10 $\varphi$ = -97.95 [rad]


KeyboardInterrupt: 

In [13]:
# print(fourier_frequencies[[np.argmin(np.abs((i*np.ones_like(fourier_frequencies))-fourier_frequencies)) for i in np.arange(0.5,3.05,.05)]])
# print(transferFunction)
# tf = np.array([
#     fourier_frequencies[[np.argmin(np.abs((i*np.ones_like(fourier_frequencies))-fourier_frequencies)) for i in np.arange(0.5,3.05,.05)]],
#     fourier_transform[[np.argmin(np.abs((i*np.ones_like(fourier_frequencies))-fourier_frequencies)) for i in np.arange(0.5,3.05,.05)]],
#     transferFunction,
#     transferFunction-fourier_transform[[np.argmin(np.abs((i*np.ones_like(fourier_frequencies))-fourier_frequencies)) for i in np.arange(0.5,3.05,.05)]]
# ])
# tf = np.transpose(tf)
transferFunction = np.append(transferFunction, [temp_transferFunction], 0)
# np.load(f'{savedata}\Full Transfer Function.npy')

In [194]:
transferFunction = np.delete(np.load(f'{savedata}\Full Transfer Function.npy'), 12,0)
transferFunction = np.delete(transferFunction, 18, 0)

fig2, (bode_amplitude, bode_phase) = plt.subplots(2,1)
fig2_manager = plt.get_current_fig_manager()
fig2_manager.set_window_title('bodeplot West Z')
fig2_manager.window.showMaximized()

bode_amplitude.sharex(bode_phase)

bode_amplitude.set_title('East Z actuator Bode plot')
bode_amplitude.set_xscale('log'); bode_amplitude.set_yscale('log')
bode_amplitude.set_ylabel('Magnitude $|H(s)|$')
bode_amplitude.set_xlabel('Frequency $[Hz]$')
bode_amplitude.grid(True,'both')
bode_amplitude.set_xlim(0.5,3.0)

bode_phase.set_ylabel('fase $arg(H(s))$  $[deg]$')
bode_phase.set_xlabel('Frequency $[Hz]$')
bode_phase.grid(True, 'both')
bode_phase.set_ylim(-275,45)

amplitudes = np.abs(transferFunction[:,3])
bode_amplitude.plot(np.abs(transferFunction[:,0]),amplitudes,'ko-', label= 'No edits')

phases = np.rad2deg(np.angle(transferFunction[:,3]))
for i, p in enumerate(phases):
    phases[i] = p if p <= 0 else p -360
bode_phase.plot(np.abs(transferFunction[:,0]), (phases), 'ko-', label='All phases above 0 lowered by 360$\degree$')

bode_amplitude.legend()
bode_phase.legend()

plt.pause(0.002)
fig2.tight_layout()

In [ ]:
# for i,trans in enumerate(transferFunction):
#     bode_amplitude.annotate(f'{i}',(trans[0],np.abs(trans[3])), textcoords="offset points", xytext=(30,10), ha='center')

# max_amp = np.max(amplitudes)
# max_amp_i = np.argmax(amplitudes)
# bode_amplitude.scatter(transferFunction[max_amp_i,0], max_amp, color='#000000')



# _3db = 10**(-3/20)*max_amp * np.ones_like(amplitudes)

# _3db_l = np.argmin(np.abs(_3db-amplitudes)[max_amp_i:])
# _3db_h = np.argmin(np.abs(_3db-amplitudes)[:max_amp_i])

# bode_amplitude.plot([transferFunction[_3db_l,0],transferFunction[_3db_h,0]], [10**(-3/20)*max_amp, 10**(-3/20)*max_amp] )

# print(_3db,'\n', _3db_l,'\n', _3db_h,'\n', max_amp_i)

In [191]:
fig3, nyquist = plt.subplots(1,1)
fig3.set_figheight(12)
fig3.set_figwidth(12)
nyquist.plot(np.real(transferFunction[:,3]),np.imag(transferFunction[:,3]))
nyquist.axvline(0, color='k')
nyquist.axhline(0, color='k')
nyquist.grid(True,'major')
nyquist.set_xlim(-0.3,0.3)
nyquist.set_ylim(-0.3,0.3)
nyquist.set_ylabel('Imaginary')
nyquist.set_xlabel(r'Real')

plt.pause(0.002)
fig3.tight_layout()
plt.show()